In [2]:
import json

json_path = '/home/thangquang/CODE/CTAI_MachineLearning/data/ise-dsc01-train_new_preprocessed.json'
data = json.load(open(json_path, "r", encoding="utf8"))

In [3]:
dataset = [ex for ex in data if ex['verdict'] != 'NEI']
len(dataset)

25384

In [4]:
dataset[0]['context']

['phát_biểu tại lễ ký_kết vào ngày 1711 , giám_đốc điều_hành kiêm đồng sáng_lập faba technology - nguyễn thành hưng nhận_định trong bối_cảnh " bình_thường mới " như hiện_tại , mô_hình đào_tạo 100 % online của funix là một trong những cách có_thể giải bài_toán khan_hiếm nhân_lực của ngành công_nghệ thông_tin .',
 'ông cũng chia_sẻ việc trở_thành đối_tác đào_tạo và tuyển_dụng cùng funix sẽ giúp faba technology đạt mục_tiêu trở_thành_công_ty offshore development chất_lượng nhất trong khu_vực .',
 'qua hợp_tác này , faba kỳ_vọng sẽ có ứng_viên chất_lượng cho rất nhiều vị_trí công_việc đang tìm_kiếm như automation tester , uiux designer , web developer , mobile developer ở nhiều cấp_độ ... " chúng_tôi hoàn_toàn tin_tưởng vào chất_lượng đào_tạo của funix " - ông nhấn_mạnh .',
 'để thu_hút và giữ_chân nhân_tài trong một thị_trường nhân_sự it khốc_liệt , faba không giới_hạn_mức lương , luôn cập_nhật để dải_lương đạt mức cạnh_tranh tốt so với thị_trường .',
 'công_ty cũng có chế_độ đãi_ngộ tốt 

In [5]:
def calc_metrics(ranks, k_list=(1,3,5)):
    """ranks: vị trí (bắt đầu từ 1) của evidence. None nếu không tìm thấy."""
    hit1   = 1 if ranks is not None and ranks==1 else 0
    mrr    = 1.0 / ranks if ranks is not None else 0.0
    recalls = {k: 1 if ranks is not None and ranks<=k else 0 for k in k_list}
    return hit1, mrr, recalls

In [6]:
metrics_sum = {
    'hit1':0,
    'mrr':0,
    'recall@3':0,
    'recall@5':0
}
N = len(dataset)

In [7]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import average_precision_score
from tqdm import tqdm
import numpy as np

In [8]:
bert_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [9]:

# for item in tqdm(dataset, desc="Evaluating"):
#     claim = item['claim'].strip()
#     context_sentences = [s.strip() for s in item['context']]
#     gold = item['evidence'].strip()
    
#     # -------- 4a.  BM25 ranking  ----------
#     tokenized_corpus = [sent.split() for sent in context_sentences]
#     bm25 = BM25Okapi(tokenized_corpus)
#     scores = bm25.get_scores(claim.split())
    
#     # -------- 4b.  SBERT ranking ----------
#     # Claim embedding
#     claim_emb = bert_model.encode(claim, convert_to_tensor=True, normalize_embeddings=True)
#     sbert_embs = bert_model.encode(context_sentences, convert_to_tensor=True,
#                                    normalize_embeddings=True)
#     cos_scores = util.cos_sim(claim_emb, sbert_embs).squeeze(0).cpu().numpy()
    
#     # Chọn baseline nào?
#     # Uncomment 1 trong 2 dòng dưới:
#     ranking_scores = scores         # BM25
#     # ranking_scores = cos_scores   # SBERT
    
#     # Xếp hạng giảm dần
#     ranked_ids = np.argsort(ranking_scores)[::-1]    # index sau sắp xếp
#     ranked_sents = [context_sentences[i] for i in ranked_ids]
    
#     # Vị trí evidence
#     rank = None
#     for idx, sent in enumerate(ranked_sents, 1):   # start=1
#         if sent == gold:
#             rank = idx
#             break
    
#     # Cộng dồn metric
#     hit1, mrr, recalls = calc_metrics(rank)
#     metrics_sum['hit1']       += hit1
#     metrics_sum['mrr']        += mrr
#     metrics_sum['recall@3']   += recalls[3]
#     metrics_sum['recall@5']   += recalls[5]

# #############################################################
# # 5. Kết quả trung bình
# #############################################################
# print("========= Baseline results =========")
# print(f"Queries evaluated      : {N}")
# print(f"Hit@1 (Accuracy)       : {metrics_sum['hit1']/N:.4f}")
# print(f"MRR                    : {metrics_sum['mrr']/N:.4f}")
# print(f"Recall@3               : {metrics_sum['recall@3']/N:.4f}")
# print(f"Recall@5               : {metrics_sum['recall@5']/N:.4f}")

In [10]:
# pip install rank_bm25 sentence-transformers underthesea tqdm
import hashlib, os, json, numpy as np, tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from underthesea import word_tokenize          # tokenizer tiếng Việt

# --------------------- 0. HÀM TIỆN ÍCH ---------------------------
def tokenize(sent: str):
    """Tách từ, trả list token (giữ dấu _)."""
    return word_tokenize(sent, format="text").split()

def hash_ctx(sents):
    return hashlib.md5(" ".join(sents).encode()).hexdigest()

def calc_single_metrics(rank):
    """Trả hit@1, mrr, recall3, recall5 cho 1 query."""
    hit1  = 1 if rank == 1 else 0
    mrr   = 1.0 / rank if rank else 0.0
    rec3  = 1 if rank and rank <= 3 else 0
    rec5  = 1 if rank and rank <= 5 else 0
    return hit1, mrr, rec3, rec5

# --------------------- 1. LOAD & CACHE ---------------------------
# new_data = [...]   # list các dict (context, claim, evidence, verdict)
dataset_raw = dataset

bert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

context_cache = {}      # cid -> dict(sentences, bm25, emb)
claims        = []      # giữ thứ tự với dataset_idx
dataset       = []      # bản ghi kèm cid

print("Pre-computing context cache ...")
for item in tqdm.tqdm(dataset_raw):
    cid = hash_ctx(item['context'])
    if cid not in context_cache:
        sents = [s.strip() for s in item['context']]
        bm25  = BM25Okapi([tokenize(s) for s in sents])
        emb   = bert.encode(sents,
                            batch_size=128,
                            convert_to_tensor=True,
                            normalize_embeddings=True)
        context_cache[cid] = {"sents": sents, "bm25": bm25, "emb": emb}
    dataset.append({**item, "cid": cid})
    claims.append(item['claim'].strip())

print("Encoding all claims ...")
claim_embs = bert.encode(claims,
                         batch_size=128,
                         convert_to_tensor=True,
                         normalize_embeddings=True)

# --------------------- 2. HÀM CHẠY CHO 1 CLAIM -------------------
def evaluate_idx(idx: int):
    ex          = dataset[idx]
    cid         = ex["cid"]
    ctx         = context_cache[cid]
    gold_sent   = ex["evidence"].strip()

    # ---- BM25 (đổi sang cos_scores nếu muốn SBERT) ---------------
    bm25_scores = ctx["bm25"].get_scores(tokenize(ex["claim"]))
    ranking     = bm25_scores            # hoặc util.cos_sim(...)
    ranked_ids  = np.argsort(ranking)[::-1]

    # ---- tìm vị trí câu evidence -------------------------------
    rank = None
    for pos, sent_id in enumerate(ranked_ids, 1):     # start at 1
        if ctx["sents"][sent_id] == gold_sent:
            rank = pos
            break
    return calc_single_metrics(rank)

# --------------------- 3. CHẠY ĐA LUỒNG --------------------------
num_workers = min(16, os.cpu_count()*2)   # ví dụ: gấp đôi core
metrics_sum = np.zeros(4, dtype=float)    # [hit1, mrr, rec3, rec5]

print(f"Evaluating with {num_workers} threads ...")
with ThreadPoolExecutor(max_workers=num_workers) as pool:
    futures = [pool.submit(evaluate_idx, i) for i in range(len(dataset))]
    for fut in tqdm.tqdm(as_completed(futures), total=len(dataset)):
        metrics_sum += np.array(fut.result())

N = len(dataset)
print("\n============= BASELINE (Thread) =============")
print(f"Queries evaluated : {N}")
print(f"Hit@1             : {metrics_sum[0]/N:.4f}")
print(f"MRR               : {metrics_sum[1]/N:.4f}")
print(f"Recall@3          : {metrics_sum[2]/N:.4f}")
print(f"Recall@5          : {metrics_sum[3]/N:.4f}")


Pre-computing context cache ...


100%|██████████| 25384/25384 [13:01<00:00, 32.49it/s]  


Encoding all claims ...
Evaluating with 16 threads ...


100%|██████████| 25384/25384 [00:04<00:00, 5401.98it/s] 


============= BASELINE (Thread) =============
Queries evaluated : 25384
Hit@1             : 0.3025
MRR               : 0.3163
Recall@3          : 0.3259
Recall@5          : 0.3315


In [11]:
# pip install rank_bm25 sentence-transformers underthesea tqdm
import hashlib, os, json, numpy as np, tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from underthesea import word_tokenize          # tokenizer tiếng Việt

# --------------------- 0. HÀM TIỆN ÍCH ---------------------------
def tokenize(sent: str):
    """Tách từ, trả list token (giữ dấu _)."""
    return word_tokenize(sent, format="text").split()

def hash_ctx(sents):
    return hashlib.md5(" ".join(sents).encode()).hexdigest()

def calc_single_metrics(rank):
    """Trả hit@1, mrr, recall3, recall5 cho 1 query."""
    hit1  = 1 if rank == 1 else 0
    mrr   = 1.0 / rank if rank else 0.0
    rec3  = 1 if rank and rank <= 3 else 0
    rec5  = 1 if rank and rank <= 5 else 0
    return hit1, mrr, rec3, rec5

# --------------------- 1. LOAD & CACHE ---------------------------
# new_data = [...]   # list các dict (context, claim, evidence, verdict)
dataset_raw = dataset

bert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

context_cache = {}      # cid -> dict(sentences, bm25, emb)
claims        = []      # giữ thứ tự với dataset_idx
dataset       = []      # bản ghi kèm cid

print("Pre-computing context cache ...")
for item in tqdm.tqdm(dataset_raw):
    cid = hash_ctx(item['context'])
    if cid not in context_cache:
        sents = [s.strip() for s in item['context']]
        bm25  = BM25Okapi([tokenize(s) for s in sents])
        emb   = bert.encode(sents,
                            batch_size=128,
                            convert_to_tensor=True,
                            normalize_embeddings=True)
        context_cache[cid] = {"sents": sents, "bm25": bm25, "emb": emb}
    dataset.append({**item, "cid": cid})
    claims.append(item['claim'].strip())

print("Encoding all claims ...")
claim_embs = bert.encode(claims,
                         batch_size=128,
                         convert_to_tensor=True,
                         normalize_embeddings=True)

# --------------------- 2. HÀM CHẠY CHO 1 CLAIM -------------------
def evaluate_idx(idx: int):
    ex          = dataset[idx]
    cid         = ex["cid"]
    ctx         = context_cache[cid]
    gold_sent   = ex["evidence"].strip()

    # ---- BM25 (đổi sang cos_scores nếu muốn SBERT) ---------------
    cos_scores  = util.cos_sim(claim_embs[idx], ctx["emb"]).squeeze(0).cpu().numpy()
    ranking     = cos_scores
    ranked_ids  = np.argsort(ranking)[::-1]

    # ---- tìm vị trí câu evidence -------------------------------
    rank = None
    for pos, sent_id in enumerate(ranked_ids, 1):     # start at 1
        if ctx["sents"][sent_id] == gold_sent:
            rank = pos
            break
    return calc_single_metrics(rank)

# --------------------- 3. CHẠY ĐA LUỒNG --------------------------
num_workers = min(16, os.cpu_count()*2)   # ví dụ: gấp đôi core
metrics_sum = np.zeros(4, dtype=float)    # [hit1, mrr, rec3, rec5]

print(f"Evaluating with {num_workers} threads ...")
with ThreadPoolExecutor(max_workers=num_workers) as pool:
    futures = [pool.submit(evaluate_idx, i) for i in range(len(dataset))]
    for fut in tqdm.tqdm(as_completed(futures), total=len(dataset)):
        metrics_sum += np.array(fut.result())

N = len(dataset)
print("\n============= BASELINE (Thread) =============")
print(f"Queries evaluated : {N}")
print(f"Hit@1             : {metrics_sum[0]/N:.4f}")
print(f"MRR               : {metrics_sum[1]/N:.4f}")
print(f"Recall@3          : {metrics_sum[2]/N:.4f}")
print(f"Recall@5          : {metrics_sum[3]/N:.4f}")


Pre-computing context cache ...


100%|██████████| 25384/25384 [13:37<00:00, 31.04it/s]  


Encoding all claims ...
Evaluating with 16 threads ...


100%|██████████| 25384/25384 [00:03<00:00, 7943.14it/s] 


============= BASELINE (Thread) =============
Queries evaluated : 25384
Hit@1             : 0.2812
MRR               : 0.2995
Recall@3          : 0.3104
Recall@5          : 0.3204
